In [ ]:
HTML(read(open("style.css"), String))

# Important Functions of Chess.jl

This notebook explains all functions we will need from the Chess.jl library to achieve our goal of implementing a Chess AI.

The library `chess.jl` can be installed and imported into the notebook by the standard package manager.

In [ ]:
using Pkg

In [ ]:
# Pkg.add("Chess")
using Chess, Chess.PGN

## Documentation
The full documentation and API Reference can be found under: https://romstad.github.io/Chess.jl/dev/

## Boards

The function `startboard()` returns a chess board in the starting position.

In [ ]:
startboard()

### Forsyth-Edwards Notation

Forsyth-Edwards Notation or short FEN is used to represent any chess position as a single String. The String is separated into # parts separated by spaces.

The first part represents the current board position. The board is viewed from white's position meaning that the top row will be the eighth row and the bottom row being the first row. Starting from the top row all pieces on this row from left to right will be notated followed by a slash `/` ending the current row and starting the new row. This is done for each of the eight rows. Pieces are notated using their initial letter of their piece name (exception the Knight using the letter `N`).

Pawn => p <br/>
Knight => n <br/>
Bishop => b <br/>
Rook => r <br/>
Queen => q <br/>
King => k <br/>

White pieces are represented using a capital letter and black with lower-case letters. Blank spaces in a row will be represented by the number of consecutive empty spaces. 

The second part is either the letter `w` for white or `b` for black depending on which sides moves next.

The third part represents the castling rights. A capital `K` means that white has the option of castling King/short side. The initial position has all castling rights and therefore has `KQkq` as the third part of the FEN String. If no castling rights are available the third part of the FEN String is represented by a `-`.

The forth part of the string is a square the moving side can En passant on. Whenever a pawn moves two squares forward the square one behind that pawn will be listed in this part of the FEN string. If there is no en passant available this part contains a `-` character.

The fifth part is the halfmove clock. It represents the number of halfmoves done without capturing a piece or advancing a pawn. It is incremented after every half move and set to 0 if one of the two event occure.

The sixth part is the fullmove counter. It shows the number of fullmoves done in the whole game. Initially it is set to 1 and is incremented after black has moved.

`fromfen(fenstring)` returns a board object given a Forsyth–Edwards Notation String inputted in the argument `fenstring`.

In [ ]:
spanishOpeningBoard = fromfen("r1bqkbnr/1ppp1ppp/p1n5/1B2p3/4P3/5N2/PPPP1PPP/RNBQK2R w KQkq - 0 4")

In [ ]:
four_knights = fromfen("r1bqkb1r/pppp1ppp/2n2n2/4p3/4P3/2N2N2/PPPP1PPP/R1BQKB1R w KQkq - 4 4")

## Move

The class `Move` represents a chess move a player can make when it is their turn. A `Move` object contains a `from` square and a `to` square. The from-square determines from where a piece is moved, likewise for the to-square. For this project we will create `Moves` by using the `movefromstring(string)` function.

### movefromstring function

The `movefromstring(string)` function takes in a string and returns a `Move` object if it can parse the string into a chess move. 


Possible strings are UCI (universal chess interface) move strings. (https://www.chessprogramming.org/UCI)

Regular Move:
- Strings specifying the from-square and to-square coordinate (e.g. "e2e4" => From e2 to e4)

Promoting:
- Strings specifying the from-square and to-square coordinate and a piece into which the pawn should promote (e.g. "a7a8q" => From a7 to a8 promoting to a queen)

If it cannot parse the string it will return `nothing`.

In [ ]:
movefromstring("e2e4")

In [ ]:
movefromstring("a7a8q")

### domove function

The `domove(board, move)` function takes in a `board` object and a `move` and returns a new board where the given `move` has been made on the given `board`. Moves an be specified using strings or Move objects. Chess.jl will try to parse the String into a move object.

In [ ]:
b = startboard()
move = movefromstring("e2e4")
domove(b, move)

In [ ]:
b = startboard()
domove(b, "e4")

### domove! function

The `domove!(board, move)` function takes in a `board` object and a `move` and applies the given `move` on the given `board` and returns an UndoInfo object. Moves can be specified using strings or Move objects. Chess.jl will try to parse the String into a move object.

In [ ]:
b = startboard()
domove!(b, "e4")
b

In [ ]:
b = startboard()
domove!(b, "e4")

### undomove! function

The `undomove!(board, UndoInfo)` takes in a board and an `UndoInfo` received from the `domove!(board, move)` function and undoes the move done in the `domove!` function and returns the previous board. 

In [ ]:
b = startboard()
u = domove!(b, "e4")
undomove!(b, u)
b

### moves function

The function `moves(board)` takes in a `board` and returns a list containing all legal moves for this board.

In [ ]:
b = startboard()
moves(b)

## Game state

The result of a game can be determined using the following methods:
- `isterminal(board)`
- `ischeckmate(board)`
- `isstalemate(board)`
- `ismaterialdraw(board)`
- `isrule50draw(board)`

### isterminal

The `isterminal(board)` function takes in a `board` and returns whether the board is in a terminal state or not. A terminal state is a position where the side to move cannot make another legal move. This is reached when the position is in a checkmate, stalemate or when there is insufficient material or when the 50 move rule has been reached.

In [ ]:
stafford_gambit_trap = fromfen("r2Bk2r/ppp2ppp/2p5/8/4n1b1/3P4/PPP1KbPP/RN1Q1B1R w kq - 2 9") # position where white is checkmated

In [ ]:
isterminal(stafford_gambit_trap)

### ischeckmate

The `ischeckmate(board)` function takes in a `board` and returns whether this board has ended in a checkmate or not.

In [ ]:
ischeckmate(stafford_gambit_trap)

### isstalemate

The `isstalemate(board)` function takes in a `board` and returns whether this board has ended in a stalemate or not.

In [ ]:
stalemate_board = fromfen("5k2/5P2/5K2/8/8/8/8/8 b - - 0 50") # position where white stalemated black

In [ ]:
isstalemate(stalemate_board)

### ismaterialdraw

The `ismaterialdraw(board)` function takes in a `board` and returns whether the board still has sufficient material to have a decisive result.

In [ ]:
insufficient_material_board = fromfen("8/8/4kb2/8/8/3K4/8/8 w - - 0 4") # position where both sides have insufficient material

In [ ]:
ismaterialdraw(insufficient_material_board)

The `isrule50draw(board)` function takes in a `board` and returns whether the 50 move rule has been reached.

In [ ]:
fifty_move_board = fromfen("8/6k1/8/8/4R3/5r2/1K6/8 b - - 100 108") # position where 50 move rule has reached

Note: fromfen is currently bugged as it does not read the 5th part of a Fen string and therefore does not read the number of halfmoves where no capture or pawn move has been done. Therefore it is initializing a new `Board` with 0 halfmoves.

In [ ]:
isrule50draw(fifty_move_board) # should be true

## Games

The `Game` and the `SimpleGame` class both store information about a whole chess game, such as the move sequence or any information about the game. The `Game` has more options than the `SimpleGame` such as branching of during the game. Therefore the `SimpleGame` performs better compared the the `Game` class.

The `Game()` function returns a `Game` object in the starting position.

In [ ]:
g = Game()

The `SimpleGame()` function returns a `Game` object in the starting position.

In [ ]:
sg = SimpleGame()

The `board(game)` takes in a `game` and returns the current position of the game as a board object.

In [ ]:
typeof(board(g))

The function `addmove` takes in a `game` and a `move` and adds the move to the given game.

In [ ]:
addmove!(g, "e4")
g

The functions `forward!(game)`, `back!(game)`, `tobeginning!(game)`, `toend!(game)` take in a `game` and move the currrent position of the game.

In [ ]:
tobeginning!(g)

In [ ]:
forward!(g)

## Pieces

The `Piece` class represents an instance of any chess piece. It holds information of the type and color of the piece. For example `PIECE_WP` for white pawn, `PIECE_BN` for black knight or `EMPTY` for an empty square. 

The class `PieceType` represents the type of the piece and therefore has the values `PAWN`, `KNIGHT`, `BISHOP`, `ROOK`, `QUEEN` and `KING`.

The class `PieceColor` represents the color a piece and is either `WHITE`, `BLACK` or `COLOR_NONE`. The `COLOR_NONE` is only used for the `EMPTY` piece.

The constructur `Piece(color, type)` takes in a `PieceColor` and a `PieceType` and returns a `Piece` object with both properties.

In [ ]:
Piece(BLACK, KING)

In [ ]:
pieceon(startboard(), FILE_A, RANK_1)

The function `ptype(piece)` takes a `piece` and returns the `PieceType` of the piece. The class `PieceType` contains the objects `PAWN`, `KNIGHT`, `BISHOP`, `ROOK`, `QUEEN` and `KING`.

In [ ]:
ptype(pieceon(startboard(), FILE_A, RANK_1))

The color of a piece is also used for the side to move on the board object.

The function `sidetomove(board)` takes in a `board` and returns the PieceColor of the side that moves next.

In [ ]:
sidetomove(startboard())

## PGN files

PGN (Portable game notation) can be used to save played games. It stores the moves that happened in the game, as well as some additional information about who the players were, the date of the game or the location of the game. 

The PGN file contains seven tags which are required. These are the `Event`, `Site`, `Date`, `Round`, `White`, `Black` and `result` Tag. Additionally there are optional tags. The optional tags are `Annotator`, `PlyCount`, `TimeControl`, `Time`, `Termination`, `Mode` and `FEN` Tag.

Using Chess.jl PGN files can be created from the `Game` or `SimpleGame` object. This way games can be exported and saved in a PGN file. These can be imported later and transformed back into a `Game` object.

### setheadervalue! function

The function `setheadervalue!(game, name, value)` takes in a `Game` or a `Simplegame`, the `name` of the header and a `value` for the header and sets these for the given `Game`. It will create a header if it has not been created or overwrites the value if it has been set.

Possible headers specify the Event, Site, Date, Round of the game as well as who played the game. 

In [ ]:
g = Game()
setheadervalue!(g, "Date", "2023.01.01")
dateplayed(g)

### PGN Reader

The constructor `PGNReader(io)` takes in an io-Stream and returns a PGNReader which will be used to interact with PGN files.

In [ ]:
io = open("./../Games/RandomChess.pgn", "r")
reader = PGNReader(io)

### readgame function

The `readgame(PGNReader)` function takes in a `PGNReader` and tries to parse the file from the io-Stream of the PGNReader into a `Game` or `SimpleGame` object. For a full implementation of this see the `PGN_Import.ipynb` notebook.

In [ ]:
readgame(reader)

### gametopgn function

The `gametopgn(g)` function takes in a `Game` or `Simplegame` and parses the game into a pgn file. Note: That this has no labels. For a complete implementation see the `PGN_Export.ipynb` notebook.

In [ ]:
g = Game()
gametopgn(g)